# How to not write slow julia code

# 0. Don't use global variables

Don't do the following

In [72]:
global parameter = 10
function k(x)
    return [x^i for i in 1:parameter]
end
k(10)

10-element Array{Int64,1}:
          10
         100
        1000
       10000
      100000
     1000000
    10000000
   100000000
  1000000000
 10000000000

##  1. Type stability

A function is *type stable*, if the input types determine the output types.

### Example

The following function is not type stable.

In [13]:
function f(x::Int)
  if x < 0
    return "negative"
  elseif x >= 0
    return "positive"
  else
    return f(x)
end
end

f (generic function with 1 method)

In [14]:
f(1), f(-1)

("positive", "negative")

How to detect type instablity?

In [15]:
@code_warntype f(1)

Variables
  #self#::Core.Compiler.Const(f, false)
  x::Int64

Body::String
1 ─ %1 = (x < 0)::Bool
└──      goto #3 if not %1
2 ─      return "negative"
3 ─ %4 = (x >= 0)::Bool
└──      goto #5 if not %4
4 ─      return "positive"
5 ─ %7 = Main.f(x)::String
└──      return %7


### Solution
Don't write type instable code.

### Example

In [20]:
global parameter = 10

function k(x)
    return [x^i for i in 1:parameter]
end
@code_warntype k(10)
@time k(100);

Variables
  #self#::Core.Compiler.Const(k, false)
  x::Int64
  #9::var"#9#10"{Int64}

Body::Array{T,N} where T where N
1 ─ %1 = Main.:(var"#9#10")::Core.Compiler.Const(var"#9#10", false)
│   %2 = Core.typeof(x)::Core.Compiler.Const(Int64, false)
│   %3 = Core.apply_type(%1, %2)::Core.Compiler.Const(var"#9#10"{Int64}, false)
│        (#9 = %new(%3, x))
│   %5 = #9::var"#9#10"{Int64}
│   %6 = (1:Main.parameter)::Any
│   %7 = Base.Generator(%5, %6)::Base.Generator{_A,var"#9#10"{Int64}} where _A
│   %8 = Base.collect(%7)::Array{T,N} where T where N
└──      return %8
  0.068430 seconds (41.60 k allocations: 2.148 MiB)


### Solution

In [22]:
function kk(x, parameter)
    return [x^i for i in 1:parameter]
end
@code_warntype kk(10, 10)
@time kk(100, 10);

Variables
  #self#::Core.Compiler.Const(kk, false)
  x::Int64
  parameter::Int64
  #13::var"#13#14"{Int64}

Body::Array{Int64,1}
1 ─ %1 = Main.:(var"#13#14")::Core.Compiler.Const(var"#13#14", false)
│   %2 = Core.typeof(x)::Core.Compiler.Const(Int64, false)
│   %3 = Core.apply_type(%1, %2)::Core.Compiler.Const(var"#13#14"{Int64}, false)
│        (#13 = %new(%3, x))
│   %5 = #13::var"#13#14"{Int64}
│   %6 = (1:parameter)::Core.Compiler.PartialStruct(UnitRange{Int64}, Any[Core.Compiler.Const(1, false), Int64])
│   %7 = Base.Generator(%5, %6)::Core.Compiler.PartialStruct(Base.Generator{UnitRange{Int64},var"#13#14"{Int64}}, Any[var"#13#14"{Int64}, Core.Compiler.PartialStruct(UnitRange{Int64}, Any[Core.Compiler.Const(1, false), Int64])])
│   %8 = Base.collect(%7)::Array{Int64,1}
└──      return %8
  0.000004 seconds (1 allocation: 160 bytes)


## 2. Variables should not change type

Similar to type stability but refers more to the internal structure of the function.

### Example

In [33]:
using Hecke

function g(x::RingElement, n)
  z = 0
  for i in 1:n
    z = z + x^i
  end
  return z
end

g (generic function with 2 methods)

In [34]:
@code_warntype g(one(ZZ), 10)

Variables
  #self#::Core.Compiler.Const(g, false)
  x::fmpz
  n::Int64
  z::Union{Int64, fmpz}
  @_5::Union{Nothing, Tuple{Int64,Int64}}
  i::Int64

Body::Union{Int64, fmpz}
1 ─       (z = 0)
│   %2  = (1:n)::Core.Compiler.PartialStruct(UnitRange{Int64}, Any[Core.Compiler.Const(1, false), Int64])
│         (@_5 = Base.iterate(%2))
│   %4  = (@_5 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_5::Tuple{Int64,Int64}::Tuple{Int64,Int64}
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = z::Union{Int64, fmpz}
│   %11 = (x ^ i)::fmpz
│         (z = %10 + %11)
│         (@_5 = Base.iterate(%2, %9))
│   %14 = (@_5 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       return z


We can ignore the yellow. But the red is pretty *bad*. The problem is that at the beginning we have `z::Int`, while in the loop is added to `x`, which turns it into the same type of `x`, that is, `z::fmpz`.

### Solution

Don't do it! Here we can write instead:

In [42]:
function gg(x::RingElement, n)
  z = zero(parent(x))
  for i in 1:n
    z = z + x^i
  end
  return x
end

gg (generic function with 2 methods)

In [41]:
R = GF(3)
x = R(1)
typeof(x) === typeof(GF(5)(1))

true

In [44]:
@code_warntype gg(x, 10)

Variables
  #self#::Core.Compiler.Const(gg, false)
  x::Nemo.gfp_elem
  n::Int64
  z::Nemo.gfp_elem
  @_5::Union{Nothing, Tuple{Int64,Int64}}
  i::Int64

Body::Nemo.gfp_elem
1 ─ %1  = Main.parent(x)::Nemo.GaloisField
│         (z = Main.zero(%1))
│   %3  = (1:n)::Core.Compiler.PartialStruct(UnitRange{Int64}, Any[Core.Compiler.Const(1, false), Int64])
│         (@_5 = Base.iterate(%3))
│   %5  = (@_5 === nothing)::Bool
│   %6  = Base.not_int(%5)::Bool
└──       goto #4 if not %6
2 ┄ %8  = @_5::Tuple{Int64,Int64}::Tuple{Int64,Int64}
│         (i = Core.getfield(%8, 1))
│   %10 = Core.getfield(%8, 2)::Int64
│   %11 = z::Nemo.gfp_elem
│   %12 = (x ^ i)::Nemo.gfp_elem
│         (z = %11 + %12)
│         (@_5 = Base.iterate(%3, %10))
│   %15 = (@_5 === nothing)::Bool
│   %16 = Base.not_int(%15)::Bool
└──       goto #4 if not %16
3 ─       goto #2
4 ┄       return x


## 3. Be explicit about the element type of arrays

### Example

In [29]:
function get_powers(x::RingElement, i::Int)
  r = []
  for j in 1:i
    push!(r, x^j)
  end
  return r[i]
end

get_powers (generic function with 1 method)

In [19]:
@code_warntype get_powers(ZZ(3), 2)

Variables
  #self#::Core.Compiler.Const(get_powers, false)
  x::fmpz
  i::Int64
  r::Array{Any,1}
  @_5::Union{Nothing, Tuple{Int64,Int64}}
  j::Int64

Body::Any
1 ─       (r = Base.vect())
│   %2  = (1:i)::Core.Compiler.PartialStruct(UnitRange{Int64}, Any[Core.Compiler.Const(1, false), Int64])
│         (@_5 = Base.iterate(%2))
│   %4  = (@_5 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_5::Tuple{Int64,Int64}::Tuple{Int64,Int64}
│         (j = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = r::Array{Any,1}
│   %11 = (x ^ j)::fmpz
│         Main.push!(%10, %11)
│         (@_5 = Base.iterate(%2, %9))
│   %14 = (@_5 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄ %18 = Base.getindex(r, i)::Any
└──       return %18


The problem is that `r = []` is the same as `r = Any[]`. Thus when accessing `r`, julia does not know what the return type will be.

In [47]:
l(x) = [ 2^x for x in 1:x]; @code_warntype l(10)

Variables
  #self#::Core.Compiler.Const(l, false)
  x::Int64
  #15::var"#15#16"

Body::Array{Int64,1}
1 ─      (#15 = %new(Main.:(var"#15#16")))
│   %2 = #15::Core.Compiler.Const(var"#15#16"(), false)
│   %3 = (1:x)::Core.Compiler.PartialStruct(UnitRange{Int64}, Any[Core.Compiler.Const(1, false), Int64])
│   %4 = Base.Generator(%2, %3)::Core.Compiler.PartialStruct(Base.Generator{UnitRange{Int64},var"#15#16"}, Any[Core.Compiler.Const(var"#15#16"(), false), Core.Compiler.PartialStruct(UnitRange{Int64}, Any[Core.Compiler.Const(1, false), Int64])])
│   %5 = Base.collect(%4)::Array{Int64,1}
└──      return %5


### Solution
Always annotate the elem type of arrays.

In [20]:
function get_powers2(x::RingElement, i::Int)
  r = typeof(x)[]
  for j in 1:i
    push!(r, x^j)
  end
  return r[i]
end

get_powers2 (generic function with 1 method)

In [21]:
@code_warntype get_powers2(ZZ(3), 2)

Variables
  #self#::Core.Compiler.Const(get_powers2, false)
  x::fmpz
  i::Int64
  r::Array{fmpz,1}
  @_5::Union{Nothing, Tuple{Int64,Int64}}
  j::Int64

Body::fmpz
1 ─ %1  = Main.typeof(x)::Core.Compiler.Const(fmpz, false)
│         (r = Base.getindex(%1))
│   %3  = (1:i)::Core.Compiler.PartialStruct(UnitRange{Int64}, Any[Core.Compiler.Const(1, false), Int64])
│         (@_5 = Base.iterate(%3))
│   %5  = (@_5 === nothing)::Bool
│   %6  = Base.not_int(%5)::Bool
└──       goto #4 if not %6
2 ┄ %8  = @_5::Tuple{Int64,Int64}::Tuple{Int64,Int64}
│         (j = Core.getfield(%8, 1))
│   %10 = Core.getfield(%8, 2)::Int64
│   %11 = r::Array{fmpz,1}
│   %12 = (x ^ j)::fmpz
│         Main.push!(%11, %12)
│         (@_5 = Base.iterate(%3, %10))
│   %15 = (@_5 === nothing)::Bool
│   %16 = Base.not_int(%15)::Bool
└──       goto #4 if not %16
3 ─       goto #2
4 ┄ %19 = Base.getindex(r, i)::fmpz
└──       return %19


Alternatively:

In [28]:
function get_powers3(x::T, i::Int) where T <: RingElement
  r = T[]
  for j in 1:i
    push!(r, x^j)
  end
  return r[i]
end

get_powers3 (generic function with 1 method)

### Example

In [24]:
function count_to_10(R::Hecke.Ring)
  r = []
  for i in 1:10
    push!(r, R(i))
  end
  return r
end

count_to_10 (generic function with 1 method)

In [25]:
@code_warntype count_to_10(ZZ)

Variables
  #self#::Core.Compiler.Const(count_to_10, false)
  R::FlintIntegerRing
  r::Array{Any,1}
  @_4::Union{Nothing, Tuple{Int64,Int64}}
  i::Int64

Body::Array{Any,1}
1 ─       (r = Base.vect())
│   %2  = (1:10)::Core.Compiler.Const(1:10, false)
│         (@_4 = Base.iterate(%2))
│   %4  = (@_4::Core.Compiler.Const((1, 1), false) === nothing)::Core.Compiler.Const(false, false)
│   %5  = Base.not_int(%4)::Core.Compiler.Const(true, false)
└──       goto #4 if not %5
2 ┄ %7  = @_4::Tuple{Int64,Int64}::Tuple{Int64,Int64}
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = r::Array{Any,1}
│   %11 = (R)(i)::fmpz
│         Main.push!(%10, %11)
│         (@_4 = Base.iterate(%2, %9))
│   %14 = (@_4 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       return r


### Solution

In [26]:
function count_to_10_better(R::Hecke.Ring)
  r = elem_type(R)[]
  for i in 1:10
    push!(r, R(i))
  end
  return r
end

count_to_10_better (generic function with 1 method)

In [27]:
@code_warntype count_to_10_better(ZZ)

Variables
  #self#::Core.Compiler.Const(count_to_10_better, false)
  R::FlintIntegerRing
  r::Array{fmpz,1}
  @_4::Union{Nothing, Tuple{Int64,Int64}}
  i::Int64

Body::Array{fmpz,1}
1 ─ %1  = Main.elem_type(R)::Core.Compiler.Const(fmpz, false)
│         (r = Base.getindex(%1))
│   %3  = (1:10)::Core.Compiler.Const(1:10, false)
│         (@_4 = Base.iterate(%3))
│   %5  = (@_4::Core.Compiler.Const((1, 1), false) === nothing)::Core.Compiler.Const(false, false)
│   %6  = Base.not_int(%5)::Core.Compiler.Const(true, false)
└──       goto #4 if not %6
2 ┄ %8  = @_4::Tuple{Int64,Int64}::Tuple{Int64,Int64}
│         (i = Core.getfield(%8, 1))
│   %10 = Core.getfield(%8, 2)::Int64
│   %11 = r::Array{fmpz,1}
│   %12 = (R)(i)::fmpz
│         Main.push!(%11, %12)
│         (@_4 = Base.iterate(%3, %10))
│   %15 = (@_4 === nothing)::Bool
│   %16 = Base.not_int(%15)::Bool
└──       goto #4 if not %16
3 ─       goto #2
4 ┄       return r


The functions `elem_type`, `parent_type`, `dense_matrix_type` help with manipulations on the type side.

In [49]:
elem_type(ZZ), parent_type(ZZ(1)), dense_matrix_type(ZZ), elem_type(FiniteField(3))

(fmpz, FlintIntegerRing, fmpz_mat, Nemo.gfp_elem)

# 4. If you have to work with instabilities, help the compiler
There are situations, where writing type unstable is unavoidable. Here is an example

In [52]:
mutable struct A
  x
end

In [60]:
R = A(2); @code_warntype R.x

Variables
  #self#::Core.Compiler.Const(getproperty, false)
  x::A
  f::Symbol

Body::Any
1 ─ %1 = Base.getfield(x, f)::Any
└──      return %1


In [58]:
function get_x(R::A)
  return R.x::Int
end

get_x (generic function with 1 method)

In [61]:
@code_warntype get_x(R)

Variables
  #self#::Core.Compiler.Const(get_x, false)
  R::A

Body::Int64
1 ─ %1 = Base.getproperty(R, :x)::Any
│   %2 = Core.typeassert(%1, Main.Int)::Int64
└──      return %2


Notice the `typeassert`!

## Random tipps
- Skim over https://docs.julialang.org/en/v1/manual/performance-tips/
- For deeply nested code (or keyword arguments), use https://github.com/JuliaDebug/Cthulhu.jl (Demo)
- If your functions allocate a lot of memory (when timed with `@btime` or `@time`), chances are high that there is type instability. Check with `@code_warntype`.